In [ ]:
# main_cd.py：用于模型训练

In [1]:
import subprocess
import os
import time
import textwrap

def get_gpu_processes():
    """获取当前运行的 GPU 进程信息"""
    result = subprocess.run(
        ['nvidia-smi', '--query-compute-apps=pid,gpu_name,used_memory', '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE,
        text=True
    )
    lines = result.stdout.strip().split('\n')
    processes = []
    for line in lines:
        parts = line.split(',')
        if len(parts) == 3:  # 确保每一行有三个值
            pid, gpu_name, used_memory = parts
            processes.append((int(pid), gpu_name, int(used_memory)))
    return processes

def get_gpu_memory_info():
    """获取 GPU 的总显存和已用显存"""
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,memory.total,memory.used', '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE,
        text=True
    )
    lines = result.stdout.strip().split('\n')
    gpu_memory_info = []
    for line in lines:
        gpu_index, total_memory, used_memory = line.split(',')
        gpu_memory_info.append((int(gpu_index), int(total_memory), int(used_memory)))
    return gpu_memory_info

def check_free_memory(gpu_memory_info, threshold_gb=30):
    """检查剩余显存是否大于指定阈值（GB）"""
    threshold_mb = threshold_gb * 1024  # 将 GB 转换为 MB
    for gpu_index, total_memory, used_memory in gpu_memory_info:
        free_memory = total_memory - used_memory
        if free_memory > threshold_mb:
            print(f"GPU {gpu_index} has {free_memory / 1024:.2f} GB free memory (threshold: {threshold_gb} GB).")
            return True, gpu_index
    return False, None

def run_train_script():
    """运行训练脚本并等待其完成"""
    script_path = "main_cd.py"
    if os.path.exists(script_path):
        print(f"Running script: {script_path}")
        # 使用阻塞模式运行脚本，确保脚本运行结束后再返回
        subprocess.run(['python', script_path], check=True)
    else:
        print(f"Script not found: {script_path}")

def get_subfolders(folder_path):
    """获取指定文件夹下的所有子文件夹"""
    if os.path.exists(folder_path):
        subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]
        return subfolders
    else:
        print(f"Folder not found: {folder_path}")
        return []

def modify_config_file(new_root_dir):
    """修改配置文件中的第一个 self.root_dir 参数，并保留原有缩进"""
    config_file_path = "/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/data_config.py"
    if os.path.exists(config_file_path):
        with open(config_file_path, 'r') as file:
            lines = file.readlines()
        
        # 修改文件内容
        with open(config_file_path, 'w') as file:
            modified = False  # 标记是否已经修改过
            for line in lines:
                if line.strip().startswith("self.root_dir") and not modified:
                    # 获取当前行的缩进
                    indent = line[:len(line) - len(line.lstrip())]
                    # 替换为新的路径，并保留缩进
                    new_line = f"{indent}self.root_dir = '../Original_Data/output/{new_root_dir}'\n"
                    file.write(new_line)
                    modified = True  # 标记为已修改
                else:
                    file.write(line)
        print(f"Modified {config_file_path}: self.root_dir = '../Original_Data/output/{new_root_dir}'")
    else:
        print(f"Config file not found: {config_file_path}")

def rename_checkpoint_folder(old_name, new_name):
    """重命名文件夹"""
    old_folder_path = f"/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/checkpoints/{old_name}"
    new_folder_path = f"/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/checkpoints/{new_name}"
    
    if os.path.exists(old_folder_path):
        os.rename(old_folder_path, new_folder_path)
        print(f"Renamed folder: {old_folder_path} -> {new_folder_path}")
    else:
        print(f"Folder not found: {old_folder_path}")

def process_folders(root_dir_list, processed_folders):
    """处理未处理的文件夹"""
    for root_dir in root_dir_list:
        if root_dir not in processed_folders:
            print(f"Processing folder: {root_dir}")
            
            # 修改配置文件
            modify_config_file(root_dir)
            
            # 运行训练脚本并等待其完成
            run_train_script()

            # 重命名文件夹
            rename_checkpoint_folder("test", root_dir)
            # 标记为已处理
            processed_folders.append(root_dir)
        else:
            print(f"Folder {root_dir} has already been processed. Skipping.")

if __name__ == "__main__":
    # 获取CDDatasets文件夹下的所有子文件夹
    cddatasets_path = "/home/public/sch005406/Anjl/ChangeDetect/Original_Data/output/"
    root_dir_list = get_subfolders(cddatasets_path)
    
    # 获取已经处理过的文件夹
    processed_folders_path = "/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/checkpoints"
    processed_folders = get_subfolders(processed_folders_path)
    
    if root_dir_list:
        print(f"Found subfolders in {cddatasets_path}: {root_dir_list}")
        print(f"Already processed folders: {processed_folders}")
        
        # 主循环：每 15 秒检测一次 GPU 显存，直到所有文件夹都处理完毕
        while root_dir_list and processed_folders != root_dir_list:
            # 获取当前运行的 GPU 进程信息
            processes = get_gpu_processes()
            print("\nCurrent GPU processes:")
            for pid, gpu_name, used_memory in processes:
                print(f"PID: {pid}, GPU: {gpu_name}, Memory Used: {used_memory} MB")
            
            # 获取 GPU 的显存信息
            gpu_memory_info = get_gpu_memory_info()
            print("\nGPU memory information:")
            for gpu_index, total_memory, used_memory in gpu_memory_info:
                free_memory = total_memory - used_memory
                print(f"GPU {gpu_index}: Total Memory: {total_memory} MB, Used Memory: {used_memory} MB, Free Memory: {free_memory} MB")
            
            # 检查剩余显存是否大于 30GB
            has_free_memory, gpu_index = check_free_memory(gpu_memory_info, threshold_gb=30)
            
            if has_free_memory:
                print(f"GPU {gpu_index} has enough free memory. Performing post-processing.")
                process_folders(root_dir_list, processed_folders)
            else:
                print("No GPU has enough free memory. Waiting for 15 seconds...")
            
            # 等待 15 秒后再次检测
            time.sleep(5)
        
        print("All folders have been processed. Exiting.")
    else:
        print(f"No subfolders found in {cddatasets_path}. Exiting.")

Found subfolders in /home/public/sch005406/Anjl/ChangeDetect/Original_Data/output/: ['VV_VH_RE4_S2_S1_RE1', 'MIRBI_VV_VH_RE4_S2_S1', 'MIRBI_VV_VH_RE4_S2_RE1', 'MIRBI_VV_VH_RE4_S1_RE1', 'MIRBI_VV_VH_S2_S1_RE1', 'NBR2_VV_VH_RE4_S2_S1', 'NBR2_VV_VH_RE4_S2_RE1', 'NBR2_VV_VH_RE4_S1_RE1', 'NBR2_VV_VH_S2_S1_RE1', 'NBR_PLUS_VV_VH_RE4_S2_S1', 'NBR_PLUS_VV_VH_RE4_S2_RE1', 'NBR_PLUS_VV_VH_RE4_S1_RE1', 'NBR_PLUS_VV_VH_S2_S1_RE1', 'MIRBI_NBR2_RE4_S2_S1_RE1', 'MIRBI_NBR_PLUS_RE4_S2_S1_RE1', 'NBR2_NBR_PLUS_RE4_S2_S1_RE1', 'RF', 'GEE_RF', 'XGBoost']
Already processed folders: ['VV_VH_RE4_S2_S1_RE1', 'MIRBI_VV_VH_RE4_S2_S1', 'MIRBI_VV_VH_RE4_S2_RE1', 'MIRBI_VV_VH_RE4_S1_RE1', 'MIRBI_VV_VH_S2_S1_RE1', 'NBR2_VV_VH_RE4_S2_S1', 'NBR2_VV_VH_RE4_S2_RE1', 'NBR2_VV_VH_RE4_S1_RE1', 'NBR2_VV_VH_S2_S1_RE1']

Current GPU processes:

GPU memory information:
GPU 0: Total Memory: 46068 MB, Used Memory: 13 MB, Free Memory: 46055 MB
GPU 0 has 44.98 GB free memory (threshold: 30 GB).
GPU 0 has enough free memory. Perfor

In [ ]:
# demo.py：用于测试模型表现效果

In [6]:
import subprocess
import os
import time

def get_gpu_processes():
    """获取当前运行的 GPU 进程信息"""
    result = subprocess.run(
        ['nvidia-smi', '--query-compute-apps=pid,gpu_name,used_memory', '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE,
        text=True
    )
    lines = result.stdout.strip().split('\n')
    processes = []
    for line in lines:
        parts = line.split(',')
        if len(parts) == 3:  # 确保每一行有三个值
            pid, gpu_name, used_memory = parts
            processes.append((int(pid), gpu_name.strip(), int(used_memory)))
    return processes

def get_gpu_memory_info():
    """获取 GPU 的总显存和已用显存"""
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,memory.total,memory.used', '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE,
        text=True
    )
    lines = result.stdout.strip().split('\n')
    gpu_memory_info = []
    for line in lines:
        gpu_index, total_memory, used_memory = line.split(',')
        gpu_memory_info.append((int(gpu_index), int(total_memory), int(used_memory)))
    return gpu_memory_info

def check_free_memory(gpu_memory_info, threshold_gb=30):
    """检查剩余显存是否大于指定阈值（GB）"""
    threshold_mb = threshold_gb * 1024  # 将 GB 转换为 MB
    for gpu_index, total_memory, used_memory in gpu_memory_info:
        free_memory = total_memory - used_memory
        if free_memory > threshold_mb:
            print(f"GPU {gpu_index} has {free_memory / 1024:.2f} GB free memory (threshold: {threshold_gb} GB).")
            return True, gpu_index
    return False, None


def run_demo_script(output_folder, project_name):
    """运行 demo.py 脚本并动态修改 --output_folder 和 --project_name 参数"""
    script_path = "demo.py"
    if os.path.exists(script_path):
        print(f"Running script: {script_path} with output_folder={output_folder}, project_name={project_name}")
        # 使用阻塞模式运行脚本，确保脚本运行结束后再返回
        subprocess.run(
            ['python', script_path, '--output_folder', output_folder, '--project_name', project_name],
            check=True
        )
    else:
        print(f"Script not found: {script_path}")


def get_subfolders(folder_path):
    """获取指定文件夹下的所有子文件夹"""
    if os.path.exists(folder_path):
        subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]
        return subfolders
    else:
        print(f"Folder not found: {folder_path}")
        return []

def modify_config_file(data_name, new_root_dir):
    """修改配置文件中的 self.root_dir 参数，并保留原有缩进"""
    config_file_path = "/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/data_config.py"
    if os.path.exists(config_file_path):
        with open(config_file_path, 'r') as file:
            lines = file.readlines()
        
        # 修改文件内容
        with open(config_file_path, 'w') as file:
            i = 0
            while i < len(lines):
                line = lines[i]
                if "quick_start" in line.strip() and i + 1 < len(lines) and lines[i + 1].strip().startswith("self.root_dir"):
                    # 获取下一行的缩进
                    indent = lines[i + 1][:len(lines[i + 1]) - len(lines[i + 1].lstrip())]
                    # 替换为新的路径，并保留缩进
                    new_line = f"{indent}self.root_dir = '../Original_Data/output/{new_root_dir}'\n"
                    file.write(line)  # 写入当前行
                    file.write(new_line)  # 写入修改后的下一行
                    i += 2  # 跳过下一行，因为已经处理过了
                else:
                    file.write(line)
                    i += 1
        print(f"Modified {config_file_path}: self.root_dir = '../Original_Data/output/{new_root_dir}' for data_name={data_name}")
    else:
        print(f"Config file not found: {config_file_path}")

def process_folders(root_dir_list, processed_folders):
    """处理未处理的文件夹"""
    for root_dir in root_dir_list:
        if root_dir not in processed_folders:
            print(f"Processing folder: {root_dir}")
            
            # 修改配置文件
            modify_config_file('quick_start', root_dir)
            
            # 设置 project_name 和 output_folder
            project_name = root_dir  # 使用文件夹名称作为 project_name
            output_folder = f"samples/predict/{root_dir}"
            
            # 运行 demo.py 脚本
            run_demo_script(output_folder, project_name)
            
            # 标记为已处理
            processed_folders.append(root_dir)
        else:
            print(f"Folder {root_dir} has already been processed. Skipping.")

if __name__ == "__main__":
    # 获取 CDDatasets 文件夹下的所有子文件夹
    cddatasets_path = "/home/public/sch005406/Anjl/ChangeDetect/Original_Data/output/"
    root_dir_list = get_subfolders(cddatasets_path)
    
    # 获取已经处理过的文件夹
    processed_folders_path = "/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/samples/predict"
    processed_folders = get_subfolders(processed_folders_path)
    
    if root_dir_list:
        print(f"Found subfolders in {cddatasets_path}: {root_dir_list}")
        print(f"Already processed folders: {processed_folders}")
        
        # 主循环：每 15 秒检测一次 GPU 显存，直到所有文件夹都处理完毕
        while root_dir_list and processed_folders != root_dir_list:
            # 获取当前运行的 GPU 进程信息
            processes = get_gpu_processes()
            print("\nCurrent GPU processes:")
            for pid, gpu_name, used_memory in processes:
                print(f"PID: {pid}, GPU: {gpu_name}, Memory Used: {used_memory} MB")
            
            # 获取 GPU 的显存信息
            gpu_memory_info = get_gpu_memory_info()
            print("\nGPU memory information:")
            for gpu_index, total_memory, used_memory in gpu_memory_info:
                free_memory = total_memory - used_memory
                print(f"GPU {gpu_index}: Total Memory: {total_memory} MB, Used Memory: {used_memory} MB, Free Memory: {free_memory} MB")
            
            # 检查剩余显存是否大于 30GB
            has_free_memory, gpu_index = check_free_memory(gpu_memory_info, threshold_gb=30)
            
            if has_free_memory:
                print(f"GPU {gpu_index} has enough free memory. Performing post-processing.")
                process_folders(root_dir_list, processed_folders)
            else:
                print("No GPU has enough free memory. Waiting for 15 seconds...")
            
            # 等待 15 秒后再次检测
            time.sleep(15)
        
        print("All folders have been processed. Exiting.")
    else:
        print(f"No subfolders found in {cddatasets_path}. Exiting.")

Found subfolders in /home/public/sch005406/Anjl/ChangeDetect/Original_Data/output/: ['VV_VH_RE4_S2_S1_RE1', 'MIRBI_VV_VH_RE4_S2_S1', 'MIRBI_VV_VH_RE4_S2_RE1', 'MIRBI_VV_VH_RE4_S1_RE1', 'MIRBI_VV_VH_S2_S1_RE1', 'NBR2_VV_VH_RE4_S2_S1', 'NBR2_VV_VH_RE4_S2_RE1', 'NBR2_VV_VH_RE4_S1_RE1', 'NBR2_VV_VH_S2_S1_RE1', 'NBR_PLUS_VV_VH_RE4_S2_S1', 'NBR_PLUS_VV_VH_RE4_S2_RE1', 'NBR_PLUS_VV_VH_RE4_S1_RE1', 'NBR_PLUS_VV_VH_S2_S1_RE1', 'MIRBI_NBR2_RE4_S2_S1_RE1', 'MIRBI_NBR_PLUS_RE4_S2_S1_RE1', 'NBR2_NBR_PLUS_RE4_S2_S1_RE1', 'RF', 'GEE_RF', 'XGBoost']
Already processed folders: ['VV_VH_RE4_S2_S1_RE1']

Current GPU processes:

GPU memory information:
GPU 0: Total Memory: 46068 MB, Used Memory: 13 MB, Free Memory: 46055 MB
GPU 0 has 44.98 GB free memory (threshold: 30 GB).
GPU 0 has enough free memory. Performing post-processing.
Folder VV_VH_RE4_S2_S1_RE1 has already been processed. Skipping.
Processing folder: MIRBI_VV_VH_RE4_S2_S1
Modified /home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/data_config

In [5]:
import os
import rasterio
from tqdm import tqdm  # 导入进度条库

# 定义路径
predict_dir = '/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/samples/predict'  # 无地理参考信息的 TIFF 文件目录
reference_dir = '/home/public/sch005406/Anjl/ChangeDetect/Original_Data/output/GEE_RF/A'  # 包含地理参考信息的 TIFF 文件目录
output_dir = '/home/public/sch005406/Anjl/ChangeDetect/BIT_CD_Tif/samples/predict_tif'  # 输出目录

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 获取所有需要处理的 TIFF 文件路径
tif_files = []
for root, dirs, files in os.walk(predict_dir):
    for file in files:
        if file.endswith('.tif'):
            tif_files.append((root, file))

# 使用 tqdm 显示进度条
for root, file in tqdm(tif_files, desc="Processing TIFF files", unit="file"):
    # 构建文件路径
    predict_tif_path = os.path.join(root, file)
    reference_tif_path = os.path.join(reference_dir, file)

    # 构建输出文件的相对路径
    relative_path = os.path.relpath(root, predict_dir)  # 获取相对于 predict_dir 的相对路径
    output_subdir = os.path.join(output_dir, relative_path)  # 构建输出子文件夹路径
    os.makedirs(output_subdir, exist_ok=True)  # 创建输出子文件夹
    output_tif_path = os.path.join(output_subdir, file)  # 构建输出文件路径

    # 检查参考文件是否存在
    if not os.path.exists(reference_tif_path):
        continue  # 跳过缺失的参考文件

    # 读取无地理参考信息的 TIFF 文件
    with rasterio.open(predict_tif_path) as src:
        predict_data = src.read(1)  # 读取第一个波段的数据
        predict_profile = src.profile  # 获取元数据

    # 读取参考 TIFF 文件的地理参考信息
    with rasterio.open(reference_tif_path) as ref:
        transform = ref.transform  # 仿射变换
        crs = ref.crs  # 坐标系

    # 更新元数据以包含地理参考信息
    predict_profile.update({
        'transform': transform,
        'crs': crs,
        'driver': 'GTiff'
    })

    # 保存带地理参考信息的 TIFF 文件
    with rasterio.open(output_tif_path, 'w', **predict_profile) as dst:
        dst.write(predict_data, 1)  # 写入数据

print("All files processed.")

Processing TIFF files: 100%|█████████████████████████████████████████████████████████████████████████████████| 2166/2166 [00:36<00:00, 59.37file/s]

All files processed.
